In [2]:
import os
os.chdir("../")

from src.chroma_store import initialize_vectorstore, load_documents_from_dir
from src.chunking_strategies import chunk_by_semantic, chunk_by_recursive_split

In [3]:

from langchain_cohere import CohereEmbeddings

embeddings = CohereEmbeddings(
    model="embed-english-v3.0",
)

In [4]:
# Load the documents from the data directory.
documents = load_documents_from_dir("data/content")

chunks = chunk_by_recursive_split(documents, chunk_size=400)
# chunks = chunk_by_semantic(documents)


--INFO-- Loading documents from data/content
--INFO-- Loaded 1 documents
Split 1 documents into 51 chunks.
Time Tracking: Advisor shall provide the Company with a written report, in a format acceptable by the Company, setting forth the number of hours in which he provided the Services, on a daily basis, as well as an aggregated monthly report at the last day of each calendar month.
{'source': 'data/content/Robinson Advisory.docx', 'start_index': 2311}


### Semantic Chunking
#### Gradient
In this method, the gradient of distance is used to split chunks along with the percentile method. This method is useful when chunks are highly correlated with each other or specific to a domain e.g. legal or medical. The idea is to apply anomaly detection on gradient array so that the distribution become wider and easy to identify boundaries in highly semantic data.

In [5]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

text_splitter = SemanticChunker(
    OpenAIEmbeddings(), breakpoint_threshold_type="interquartile"
)

In [6]:
docs = text_splitter.create_documents([document.page_content for document in documents])
print(docs[2].page_content)

2024-07-07 22:11:45 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


(c) Work Product: Any product of the Services, tangible and/or intangible, including any inventions, discoveries and/or other intellectual property. (d) Confidential Information: any information disclosed by the Company to the Advisor and/or created as a result of the Services (including Work Product), of any type, form (including orally) or media, including (without limitation), code, specifications, architecture, design, data, algorithms, business plans, budget, customers / suppliers lists etc.), but excluding information which prior to the disclosure was publicly available or known to the Advisor (both without breach of any confidentiality obligation towards the Company). Use: The Advisor may use the Confidential Information only for the purpose of providing the Services and shall not obtain any rights in it. The Advisor shall stop using Confidential Information and/or return it to the Company and/or destroy it immediately upon Company’s request. The Advisor may disclose Confidentia

In [7]:
vectorstore = initialize_vectorstore(docs)

2024-07-07 22:11:48 - src.chroma_store - INFO - Clearing out the chroma database.
2024-07-07 22:11:48 - src.chroma_store - INFO - Creating a new chroma database.
2024-07-07 22:11:49 - chromadb.telemetry.product.posthog - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-07-07 22:11:51 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


### Initialize retriever

In [ ]:
similarity_threshold = 0.5
similarity_count = 20
retriever = vectorstore.as_retriever(search_type="similarity_score_threshold",
                                      search_kwargs={'score_threshold': similarity_threshold,
                                                      "k": similarity_count})